In [1]:
import cv2
import numpy as np
from ipywidgets import widgets, interact, interactive_output, HBox, VBox
from matplotlib import pyplot as plt
%matplotlib widget

In [2]:
def prepare_checker_img(H=256,W=256,s=16, plot_img:bool=False):
    img = np.zeros((H,W), np.float32)
    for iy in range(0,H,s):
        for ix in range(0,W,2*s):
            if (iy//s)%2==0:
                img[iy:iy+s,ix:ix+s] = 1.0
            else:
                img[iy:(iy+s),(ix+s):(ix+2*s)] = 1.0

    if plot_img:
        plt.imshow(img),plt.show()
        pass
    return img


# prepare_checker_img(plot_img=True)

In [ ]:
img = prepare_checker_img()
h,w = img.shape
mtx = np.eye(3)
mtx[0,2] = w/2
mtx[1,2] = h/2

mtx[0,0] = 1
mtx[1,1] = 0.9

dist = (0.00001,0,0,0)
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w-1,h-1), 1, (w,h))
#getOptimalNewCameraMatrix is causing 1 pixel shrink somehow. better to avoid it.

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

fig,axs=plt.subplots(1,3,sharex=True, sharey=True)
axs[0].imshow(img)
axs[1].imshow(dst)
axs[2].imshow(dst-img)
plt.show()


In [8]:
class Calib:
    def __init__(self) -> None:
        self.img = prepare_checker_img()
        self.out1 = widgets.Output(layout = {"border": "1px solid black"})
        with self.out1:
            self.fig,self.axs=plt.subplots(2,3,sharex=True, sharey=True)
        
    def undistort(self, fx,fy,cx,cy,k1,k2,k3,p1,p2):
        mtx = np.array([[fy,0,cx],[0,fx,cy],[0,0,1]], np.float32)
        dist= (k1,k2,p1,p2,k3)
        # dst = cv2.undistort(img, mtx, dist, None, mtx)
        mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, mtx, (w,h), 5)
        dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
    
        self.out1.clear_output()
        with self.out1:
            self.axs[0,0].imshow(img)
            self.axs[0,1].imshow(dst)
            self.axs[0,2].imshow(dst-img)
            self.axs[1,0].imshow(mapx)
            self.axs[1,1].imshow(mapy)
            plt.show()

calib =Calib()

fx=widgets.FloatSlider(min=0.5, max=10, step=0.1, value=1.0, description='fx')
fy=widgets.FloatSlider(min=0.5, max=10, step=0.1, value=1.0, description='fy')
cx=widgets.FloatSlider(min=0, max=w, step=w/20, value=w/2, description='cx')
cy=widgets.FloatSlider(min=0, max=h, step=h/20, value=h/2, description='cy')
k1=widgets.FloatSlider(min=-0.00005, max=0.00005, step=0.000001, value=0.0, description='k1')
k2=widgets.FloatSlider(min=-0.000005, max=0.000005, step=0.000000001, value=0.0, description='k2')
k3=widgets.FloatSlider(min=-0.00005, max=0.00005, step=0.000001, value=0.0, description='k3')
p1=widgets.FloatSlider(min=-0.00005, max=0.00005, step=0.000001, value=0.0, description='p1')
p2=widgets.FloatSlider(min=-0.00005, max=0.00005, step=0.000001, value=0.0, description='p2')


out = interactive_output(calib.undistort, {'fx':fx,'fy':fy,'cx':cx,'cy':cy,'k1':k1,'k2':k2,'p1':p1,'p2':p2,'k3':k3})
display(VBox([HBox([fx,fy,cx,cy]),HBox([k1,k2,p1,p2,k3])]), calib.out1)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
def initUndistortRectifyMap(mtx, dist, size):
    u,v = np.meshgrid(np.arange(size[1]), np.arange(size[0]))
    k4,k5,k6 = 0,0,0
    k1,k2,p1,p2,k3 =dist
    x = (u-cx)/fx
    y = (v-cy)/fy
    x_1,y_1 = x,y
    x2 = x_1**2
    y2 = y_1**2
    r2 = x2 + y2
    
    r4 = r2*r2
    r6 = r2*r4

    dist_k_top = 1 + k1*r2 + k2*r4 + k3*r6
    dist_k_bot = 1 + k4*r2 + k5*r4 + k6*r6
    dist_k = dist_k_top/dist_k_bot
    x_2 = x_1*(dist_k) + 2*p1*x_1*y_1 + p2*(r2 + 2*x2)
    y_2 = y_1*(dist_k) + p1*(r2 + 2*y2) + 2*p2*x_1*y_1

    x_3 = x_2
    y_3 = y_2
    out_x = x_3*fx + cx
    out_y = y_3*fy + cy
    return out_x, out_y


def test_initUndistortRectifyMap():
    fx,fy = 1,1
    cx,cy = w//2, h//2
    k1,k2,k3 = 0.00005,0.0,0.0
    p1,p2 = 0.0,0.0
    mtx = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]], dtype=np.float32)
    dist = (k1,k2,p1,p2,k3)
    mapx_gt,mapy_gt = cv2.initUndistortRectifyMap(mtx, dist,None, mtx, (w,h), 5)

    mapx, mapy = initUndistortRectifyMap(mtx, dist, (w,h))

    fig,axs = plt.subplots(2,3,sharex=True, sharey=True)
    axs[0,0].imshow(mapx)
    axs[0,1].imshow(mapx_gt)
    axs[0,2].imshow(mapx-mapx_gt)
    axs[1,0].imshow(mapy)
    axs[1,1].imshow(mapy_gt)
    axs[1,2].imshow(mapy-mapy_gt)
    plt.show()

test_initUndistortRectifyMap()

TypeError: unsupported operand type(s) for -: 'int' and 'FloatSlider'